In [1]:
from numpy import *
from scipy import *
import numpy as np
from matplotlib import *

from scipy import integrate
from scipy import interpolate
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

import xlrd

rc('text',usetex = True)
rc('font', family='serif',size = 12)

In [23]:
def R4(X): 
    R2,R3,B = X
    return (1.27708795*(R2**-1.18177842)*(R3**2.14218804)*(B**0.70136265))

In [24]:
def Qc(X): 
    P1,P2,P3,P4,P5= X
    return (2.22068098*P1+0.52868254*P2+-2.22664497*P3+-0.02146037*P4+-5.09505357*P5)

In [25]:
def Qh(X): 
    P1,P2,P3,P4,P5= X
    return (2.77459330e+00*P1+1.28933075e+00*P2-2.79557285e+00*P3+1.29594714e-03*P4-4.39735607e+00*P5)

In [26]:
def W_Hex(X):
    P1,P2 = X
    return (a*P1**b + c*P2**d)

In [22]:
def Prop_param(X):
    T_c,T_h,B = X
    C_H = 1.65161038/100*T_c+2.59946197*T_h-1.14324309*100*B
    C_L = 10.16443738*T_c-5.32568341*T_h-0.71305997*B
    dT_mg = -0.01057708*T_c+0.01163704*T_h+1.73509682*B
    dT_dmg = -0.01229483*T_c+0.01376157*T_h+0.76454655*B
    CdT_mg = -9.517569*T_c+11.49711993*T_h+820.04168503*B
    CdT_dmg = 4.39480124*T_c-7.07931118/10*T_h+9.15928168*100*B
    return(C_H,C_l,dT_mg,dT_dmg,CdT_mg,CdT_dmg)

In [7]:
##Properties of the solid 

rho_s = 7900  #[kg/m3] Density of the solid
k_s   = 8 #[W/(m2.K)] #Thermal conductivity of the solid

## Propertis of the fluid

rho_f = 1000  #[kg/m3] Density of the fluid
c_f   = 4181  #[kJ/(kg.K)] # Specific heat of the fluid
Pr_f  = 6.136 #[-] Prandtl of the fluid
mu_f  = 0.0008901 #[kg/m-2] #Viscosity of the fluid
k_f   = 0.6065  #[W/(m2.K)] #Thermal conductivity of the fluid

## Porous medium
d_p = 500*10**(-6) #[m] Particle diameter
Epsilon = 0.45  #[-] Porosity

#Magnetic Circuit
B_rem = 1.35
N_p = 2
N_Rp = 2

Calculation

In [16]:
#Porous medium
m_s   = L*W*H*10**-9*(1-Epsilon)*rho_s #mass of the AMR
Beta = (1-Epsilon)*6/d_p
v_s   = (m_f/3600)/(rho_f*W*H*10**-6) #Superficial velocity

#Dimensionless Numbers
Re_dp = d_p*v_s*rho_f/((1-Epsilon)*mu_f) #Reynolds number
Nu = 2*(1+4*(1-Epsilon)/Epsilon)+((1-Epsilon)**0.5)*Re_dp**0.6*Pr_f**(1/3) # Nusselt number
Pe = Re_dp*Pr_f

h_int = Nu*k_f/d_p
dP = L*(150*(1-Epsilon)**2*mu_f*v_s/(Epsilon**3*d_p**2) + 1.75*(1-Epsilon)*rho_f*v_s**2/(Epsilon**3*d_p))


#Effective conduction
k_e_f = k_f*Epsilon #Fluid effective conduction


a_0 = exp(-1.084-6.778*(Epsilon-0.298))
f_0 = 0.8
k_e_s = k_f*((1-a_0)*(Epsilon*f_0+(1-Epsilon*f_0)*k_s/k_f)/(1-Epsilon*(1-f_0)+k_s/k_f*Epsilon*(1-f_0))+a_0*(2*((k_s/k_f)**2)*(1-Epsilon)+(1+2*Epsilon)*k_s/k_f)/((2+Epsilon)*k_s/k_f+(1-Epsilon))) #Solid effective conduction

D_ = (k_f*rho_f/c_f)*0.75*Pe/2

k_s_eff = k_e_s
k_f_eff = k_e_f + rho_f*c_f*D_

NameError: name 'L' is not defined

In [14]:
NTU_f  = h_int*Beta*(L*W*H*10**(-9))/(m_f/3600*c_f) 


Eff = Data.sheet_by_name('Eff') 

Effect = []

for i in range(7):
     Effect.append(array(Eff.col_values(i)))

phi = np.arange(0.1,2.01,0.1)
NTU = np.arange(50,351,50)
NN,PP = np.meshgrid(NTU,phi)

Effectiv = interpolate.interp2d(phi,NTU,Effect,kind = 'linear')

NameError: name 'h_int' is not defined

In [ ]:
Pi_6 = ((m_f)*c_f)/(m_s*C_L*f)

Efness_HB = zeros(len(Pi_6))
Efness_CB = zeros(len(Pi_6))

for i in range(len(Pi_6)):
    Efness_HB[i] = Effectiv(Pi_6[i]/2,NTU_f[i])
    Efness_CB[i] = Effectiv(Pi_6[i]/(2*Pi_4[i]),NTU_f[i])


Q_pas_CB = Efness_CB*(T_h - T_c)*m_f*FE*c_f
Q_act_CB = dT_mg*m_f*c_f*FE

Q_pas_HB = Efness_HB*(T_h - T_c)*m_f*FE*c_f
Q_act_HB = dT_dmg*m_f*c_f*FE

Q_span = (T_h - T_c)*m_f*c_f*FE
Q_cond = ((1-Epsilon)*k_s_eff + Epsilon*k_f_eff)*(W*H*10**-6)*(T_h-T_c)/L
W_mag  = m_s*f*(CdT_mg - CdT_dmg)

### Closure Relations

Regenerator arranges

In [9]:
N_arr = 1/FE
N_r = N_Rp*N_p*N_arr

NameError: name 'FE' is not defined

Heat Exchangers

In [ ]:
ECr_h = Q_h/(m_f*c_f*(T_he - T_h))
ECr_c = Q_c/(m_f*c_f*(T_c - T_ce))